In [1]:
# ============================================================================
# NOTEBOOK 6: INTEGRAÇÃO TEMPORAL - BASE t0 + PARQUETS t1
# ============================================================================
# Objetivo: Consolidar base histórica (t0) com snapshots semanais (Parquets)
#           gerando colunas First Detected Date e Last Detected Date
#
# Inputs:
#   - Base_t0.xlsx (base consolidada até agosto/2024)
#   - dados_catalogo/staging/*.parquet (julho/2024 → agosto/2025)
#
# Output:
#   - Base_Consolidada_t0_t1_YYYY-MM-DD.xlsx
# ============================================================================


In [2]:
# ----------------------------------------------------------------------------
# CÉLULA 1: Imports e Configuração
# ----------------------------------------------------------------------------
import pandas as pd
import pathlib
import duckdb
from datetime import datetime, timedelta
import numpy as np
from tqdm.auto import tqdm

print("✅ Bibliotecas importadas com sucesso")
print(f"🕒 Início do processamento: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


✅ Bibliotecas importadas com sucesso
🕒 Início do processamento: 2025-10-13 17:56:56


In [3]:
# ----------------------------------------------------------------------------
# CÉLULA 2: Definição de Constantes e Mapeamentos
# ----------------------------------------------------------------------------

# === CAMINHOS ===
ARQUIVO_T0 = pathlib.Path('Base tratada 2024.xlsx')  
PASTA_PARQUETS = pathlib.Path('dados_catalogo/staging')
CAMINHO_PARQUETS = str(PASTA_PARQUETS / '*.parquet')
PASTA_OUTPUT = pathlib.Path('dados_catalogo/processed')
PASTA_OUTPUT.mkdir(exist_ok=True, parents=True)

# === CONSTANTES ===
DATA_CORTE = '30/08/2024'  # Data de referência para fechamento
GAP_TOLERANCIA_DIAS = 22   # Tolerância para considerar períodos distintos
DATA_INICIO_OVERLAP = '11/07/2024'  # Início do período de overlap

# === MAPEAMENTO DE PACKAGE ===
MAPEAMENTO_PACKAGE = {
    'TVOD': 'Transaction VOD',
    'SVOD': 'Subscription VOD',
    'Validated VOD': 'Validated VOD',
    'TV Everywhere': 'TV Everywhere',
    'Free VOD': 'Advertising VOD/Free VOD'
    }

# Criar dicionário reverso para normalização
PACKAGE_NORMALIZADO = {
    # === SVOD ===
    'svod': 'SVOD',
    'subscription': 'SVOD',
    'subscription vod': 'SVOD',
    
    # === TVOD ===
    'tvod': 'TVOD',
    'transactional': 'TVOD',
    'transaction vod': 'TVOD',
    
    # === Free VOD ===
    'free vod': 'Free VOD',
    'free': 'Free VOD',
    'free/free with ads': 'Free VOD',
    'advertising vod/free vod': 'Free VOD',
    
    # === TV Everywhere ===
    'tv everywhere': 'TV Everywhere',
    
    # === Validated VOD ===
    'validated vod': 'Validated VOD'
}

for xlsx_val, parquet_val in MAPEAMENTO_PACKAGE.items():
    PACKAGE_NORMALIZADO[xlsx_val.lower().strip()] = xlsx_val
    PACKAGE_NORMALIZADO[parquet_val.lower().strip()] = xlsx_val

print("✅ Constantes e mapeamentos definidos")
print(f"   📂 Base t0: {ARQUIVO_T0}")
print(f"   📂 Parquets: {CAMINHO_PARQUETS}")
print(f"   📅 Data de corte: {DATA_CORTE}")
print(f"   ⏱️  Gap tolerância: {GAP_TOLERANCIA_DIAS} dias")



✅ Constantes e mapeamentos definidos
   📂 Base t0: Base tratada 2024.xlsx
   📂 Parquets: dados_catalogo\staging\*.parquet
   📅 Data de corte: 30/08/2024
   ⏱️  Gap tolerância: 22 dias


In [4]:
# ----------------------------------------------------------------------------
# CÉLULA 3: Carregamento da Base t0
# ----------------------------------------------------------------------------
print("\n" + "="*70)
print("📥 CARREGANDO BASE t0")
print("="*70)

# Carregar base t0
df_t0 = pd.read_excel(ARQUIVO_T0)

print(f"✅ Base t0 carregada: {len(df_t0):,} linhas x {len(df_t0.columns)} colunas")

# Renomear colunas para padrão dos Parquets
mapeamento_colunas_t0 = {
    'Platform': 'Platform_Name',
    'Plan/Channel': 'Channel',
    'First Detected Date': 'First_Detected',
    'Last Detected Date': 'Last_Detected'
}

df_t0 = df_t0.rename(columns=mapeamento_colunas_t0)

# Verificar colunas essenciais
colunas_essenciais = ['UID', 'Platform_Name', 'Package', 'Channel', 'First_Detected', 'Last_Detected']
colunas_faltando = [col for col in colunas_essenciais if col not in df_t0.columns]

if colunas_faltando:
    print(f"⚠️  ATENÇÃO: Colunas faltando na t0: {colunas_faltando}")
    print("   Verifique os nomes das colunas e ajuste o mapeamento acima.")
else:
    print("✅ Todas as colunas essenciais presentes")




📥 CARREGANDO BASE t0
✅ Base t0 carregada: 383,288 linhas x 28 colunas
✅ Todas as colunas essenciais presentes


In [5]:
# ----------------------------------------------------------------------------
# CÉLULA 3.1: Criar chaves
# ----------------------------------------------------------------------------
# Criar chave única
df_t0['chave'] = (
    df_t0['UID'].astype(str) + '|' + 
    df_t0['Platform_Name'].astype(str) + '|' + 
    df_t0['Package'].fillna('').astype(str) + '|' + 
    df_t0['Channel'].fillna('').astype(str)
)

print(f"✅ Chaves únicas criadas: {df_t0['chave'].nunique():,} combinações distintas")

# Converter datas para datetime (formato dd/mm/yyyy)
df_t0['First_Detected'] = pd.to_datetime(df_t0['First_Detected'], format='%d/%m/%Y', errors='coerce')
df_t0['Last_Detected'] = pd.to_datetime(df_t0['Last_Detected'], format='%d/%m/%Y', errors='coerce')

print(f"✅ Datas convertidas para datetime")

✅ Chaves únicas criadas: 329,122 combinações distintas
✅ Datas convertidas para datetime


In [6]:
# ----------------------------------------------------------------------------
# CÉLULA 4: Carregamento e Preparação dos Parquets (DuckDB)
# ----------------------------------------------------------------------------
print("\n" + "="*70)
print("📥 CARREGANDO PARQUETS COM DUCKDB")
print("="*70)

con = duckdb.connect(database=':memory:')

# Query para carregar todos os Parquets
query_load_parquets = f"""
SELECT 
    "BB_UID" as UID,
    "Platform_Name",
    "Package",
    "Channel",
    "data_ref"
FROM read_parquet('{CAMINHO_PARQUETS}', union_by_name=true)
WHERE "BB_UID" IS NOT NULL
"""

print("🔄 Executando query DuckDB...")
df_parquets_raw = con.execute(query_load_parquets).fetch_df()

print(f"✅ Parquets carregados: {len(df_parquets_raw):,} registros")
print(f"   📅 Período: {df_parquets_raw['data_ref'].min()} → {df_parquets_raw['data_ref'].max()}")
print(f"   🎯 UIDs únicos: {df_parquets_raw['UID'].nunique():,}")

# Converter data_ref para datetime
df_parquets_raw['data_ref'] = pd.to_datetime(df_parquets_raw['data_ref'])



📥 CARREGANDO PARQUETS COM DUCKDB
🔄 Executando query DuckDB...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ Parquets carregados: 20,985,955 registros
   📅 Período: 2024-07-11 → 2025-08-28
   🎯 UIDs únicos: 301,413


In [7]:
# ----------------------------------------------------------------------------
# CÉLULA 5: Normalização de Package
# ----------------------------------------------------------------------------
print("\n" + "="*70)
print("🔧 NORMALIZANDO CAMPO PACKAGE")
print("="*70)

def normalizar_package(package_value):
    """Normaliza valores de Package usando o mapeamento definido"""
    if pd.isna(package_value):
        return ''
    
    valor_limpo = str(package_value).lower().strip()
    return PACKAGE_NORMALIZADO.get(valor_limpo, str(package_value))

# Aplicar normalização
df_t0['Package_normalizado'] = df_t0['Package'].apply(normalizar_package)
df_parquets_raw['Package_normalizado'] = df_parquets_raw['Package'].apply(normalizar_package)

print("✅ Package normalizado")

# Recriar chave com Package normalizado
df_t0['chave'] = (
    df_t0['UID'].astype(str) + '|' + 
    df_t0['Platform_Name'].astype(str) + '|' + 
    df_t0['Package_normalizado'].fillna('').astype(str) + '|' + 
    df_t0['Channel'].fillna('').astype(str)
)

df_parquets_raw['chave'] = (
    df_parquets_raw['UID'].astype(str) + '|' + 
    df_parquets_raw['Platform_Name'].astype(str) + '|' + 
    df_parquets_raw['Package_normalizado'].fillna('').astype(str) + '|' + 
    df_parquets_raw['Channel'].fillna('').astype(str)
)

# Verificar distribuição de Package nos Parquets
print("\n📊 Distribuição de Package nos Parquets (top 10):")
print(df_parquets_raw['Package_normalizado'].value_counts().head(10))



🔧 NORMALIZANDO CAMPO PACKAGE
✅ Package normalizado

📊 Distribuição de Package nos Parquets (top 10):
Package_normalizado
SVOD             7396673
Free VOD         7236428
TVOD             3448681
TV Everywhere    2700449
Validated VOD     203724
Name: count, dtype: int64


In [8]:
# ----------------------------------------------------------------------------
# CÉLULA 6: Função de Detecção de Gaps (22 dias)
# ----------------------------------------------------------------------------

def detectar_periodos_com_gap(datas_serie, tolerancia_dias=22):
    """
    Detecta períodos distintos em uma série de datas, considerando gap de tolerância.
    
    Args:
        datas_serie: pandas Series com datas
        tolerancia_dias: gap em dias para considerar novo período
        
    Returns:
        lista de dicionários com {'inicio': data, 'fim': data}
    """
    if len(datas_serie) == 0:
        return []
    
    datas_ordenadas = sorted(datas_serie.dropna().unique())
    
    if len(datas_ordenadas) == 0:
        return []
    
    periodos = []
    periodo_atual = {'inicio': datas_ordenadas[0], 'fim': datas_ordenadas[0]}
    
    for i in range(1, len(datas_ordenadas)):
        data_anterior = datas_ordenadas[i-1]
        data_atual = datas_ordenadas[i]
        
        gap_dias = (data_atual - data_anterior).days
        
        if gap_dias <= tolerancia_dias:
            # Continua no mesmo período
            periodo_atual['fim'] = data_atual
        else:
            # Gap excedeu tolerância - fecha período atual e abre novo
            periodos.append(periodo_atual.copy())
            periodo_atual = {'inicio': data_atual, 'fim': data_atual}
    
    # Adicionar último período
    periodos.append(periodo_atual)
    
    return periodos

print("✅ Função de detecção de gaps definida")


✅ Função de detecção de gaps definida


In [19]:
# ----------------------------------------------------------------------------
# CÉLULA 7: Processamento Grupo 1 - Obras da t0 (OTIMIZADO COM DUCKDB)
# ----------------------------------------------------------------------------
print("\n" + "="*70)
print("🔄 PROCESSANDO GRUPO 1: OBRAS DA BASE t0 (OTIMIZADO)")
print("="*70)

# Registrar t0 no DuckDB
con.register('df_t0_reg', df_t0)
con.register('df_parquets_reg', df_parquets_raw)

print(f"📊 Total de linhas na t0: {len(df_t0):,}")
print("🚀 Processando com DuckDB (vetorizado)...")

# Query SQL para processar Grupo 1
query_grupo1 = """
WITH 
-- 1. Aparições nos Parquets para chaves da t0
aparicoes_t0 AS (
    SELECT 
        p.chave,
        p.data_ref,
        t.First_Detected as first_t0,
        t.Last_Detected as last_t0
    FROM df_parquets_reg p
    INNER JOIN df_t0_reg t ON p.chave = t.chave
),

-- 2. Detectar gaps usando window function
gaps_detectados AS (
    SELECT 
        chave,
        data_ref,
        first_t0,
        last_t0,
        LAG(data_ref) OVER (PARTITION BY chave ORDER BY data_ref) as data_anterior,
        DATEDIFF('day', 
            LAG(data_ref) OVER (PARTITION BY chave ORDER BY data_ref), 
            data_ref
        ) as gap_dias
    FROM aparicoes_t0
),

-- 3. Atribuir período baseado em gaps > 22 dias
periodos_numerados AS (
    SELECT 
        chave,
        data_ref,
        first_t0,
        last_t0,
        SUM(CASE WHEN gap_dias > 22 OR gap_dias IS NULL THEN 1 ELSE 0 END) 
            OVER (PARTITION BY chave ORDER BY data_ref 
                  ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as periodo_id
    FROM gaps_detectados
),

-- 4. Agregar por chave + período
periodos_agregados AS (
    SELECT 
        chave,
        periodo_id,
        MIN(first_t0) as first_t0,
        MAX(last_t0) as last_t0,
        MIN(data_ref) as first_parquet,
        MAX(data_ref) as last_parquet,
        COUNT(*) as num_aparicoes
    FROM periodos_numerados
    GROUP BY chave, periodo_id
)

SELECT 
    chave,
    periodo_id,
    first_t0,
    last_t0,
    first_parquet,
    last_parquet,
    num_aparicoes,
    -- Decidir First final: usar t0 se for anterior aos Parquets
    CASE 
        WHEN first_t0 < first_parquet THEN first_t0
        ELSE first_parquet
    END as First_Detected_final,
    -- Decidir Last final: NULL se último período está recente
    CASE 
        WHEN last_parquet >= '2025-07-01' THEN NULL
        ELSE last_parquet
    END as Last_Detected_final
FROM periodos_agregados
"""

df_grupo1_parquets = con.execute(query_grupo1).fetch_df()

print(f"✅ Obras da t0 com aparições nos Parquets: {len(df_grupo1_parquets):,} períodos")

# Agora buscar obras da t0 que NÃO aparecem nos Parquets (mantêm dados originais)
query_t0_sem_parquets = """
SELECT DISTINCT t.*
FROM df_t0_reg t
LEFT JOIN df_parquets_reg p ON t.chave = p.chave
WHERE p.chave IS NULL
"""

df_grupo1_sem_parquets = con.execute(query_t0_sem_parquets).fetch_df()
print(f"✅ Obras da t0 SEM aparições nos Parquets: {len(df_grupo1_sem_parquets):,} linhas")

# Para as que têm aparições, fazer join para trazer metadados completos
df_grupo1_com_metadados = df_grupo1_parquets.merge(
    df_t0[['chave', 'UID', 'Platform_Name', 'Package', 'Channel']].drop_duplicates(),
    on='chave',
    how='left'
)

# Renomear colunas finais
df_grupo1_com_metadados = df_grupo1_com_metadados.rename(columns={
    'First_Detected_final': 'First_Detected',
    'Last_Detected_final': 'Last_Detected'
})

# Consolidar Grupo 1
df_grupo1 = pd.concat([
    df_grupo1_com_metadados[['UID', 'Platform_Name', 'Package', 'Channel', 'First_Detected', 'Last_Detected', 'chave']],
    df_grupo1_sem_parquets[['UID', 'Platform_Name', 'Package', 'Channel', 'First_Detected', 'Last_Detected', 'chave']]
], ignore_index=True)

print(f"✅ Grupo 1 TOTAL: {len(df_grupo1):,} linhas")
print(f"   📊 Chaves únicas: {df_grupo1['chave'].nunique():,}")



🔄 PROCESSANDO GRUPO 1: OBRAS DA BASE t0 (OTIMIZADO)
📊 Total de linhas na t0: 383,288
🚀 Processando com DuckDB (vetorizado)...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ Obras da t0 com aparições nos Parquets: 310,149 períodos
✅ Obras da t0 SEM aparições nos Parquets: 166,499 linhas
✅ Grupo 1 TOTAL: 476,648 linhas
   📊 Chaves únicas: 329,122


In [28]:
# ----------------------------------------------------------------------------
# DIAGNÓSTICO GRUPO 1: Por que 476K linhas?
# ----------------------------------------------------------------------------
print("\n" + "="*70)
print("🔍 DIAGNÓSTICO: INVESTIGANDO GRUPO 1")
print("="*70)

# Criar chave temp
df_grupo1['chave_temp'] = (
    df_grupo1['UID'].astype(str) + '|' + 
    df_grupo1['Platform_Name'].astype(str) + '|' + 
    df_grupo1['Package'].fillna('').astype(str) + '|' + 
    df_grupo1['Channel'].fillna('').astype(str)
)

df_t0['chave_temp'] = (
    df_t0['UID'].astype(str) + '|' + 
    df_t0['Platform_Name'].astype(str) + '|' + 
    df_t0['Package_normalizado'].fillna('').astype(str) + '|' + 
    df_t0['Channel'].fillna('').astype(str)
)

# Comparar número de linhas por chave: t0 vs Grupo1
linhas_t0_por_chave = df_t0.groupby('chave_temp').size()
linhas_g1_por_chave = df_grupo1.groupby('chave_temp').size()

# Chaves que aumentaram
comparacao = pd.DataFrame({
    't0': linhas_t0_por_chave,
    'grupo1': linhas_g1_por_chave
}).fillna(0)

comparacao['diferenca'] = comparacao['grupo1'] - comparacao['t0']
chaves_aumentaram = comparacao[comparacao['diferenca'] > 0].sort_values('diferenca', ascending=False)

print(f"\n📊 Chaves que AUMENTARAM linhas: {len(chaves_aumentaram):,}")
print(f"📊 Total de linhas extras criadas: {chaves_aumentaram['diferenca'].sum():,.0f}")

print("\n🔴 Top 10 chaves com mais linhas extras:")
for idx, (chave, row) in enumerate(chaves_aumentaram.head(10).iterrows(), 1):
    print(f"\n{idx}. Chave: {chave[:80]}...")
    print(f"   t0: {row['t0']:.0f} linhas → Grupo1: {row['grupo1']:.0f} linhas (+ {row['diferenca']:.0f})")
    
    # Mostrar períodos da t0
    print("   Períodos na t0:")
    periodos_t0 = df_t0[df_t0['chave_temp'] == chave][['First_Detected', 'Last_Detected']]
    for _, p in periodos_t0.iterrows():
        first_str = p['First_Detected'].strftime('%d/%m/%Y') if pd.notna(p['First_Detected']) else 'vazio'
        last_str = p['Last_Detected'].strftime('%d/%m/%Y') if pd.notna(p['Last_Detected']) else 'vazio'
        print(f"      {first_str} → {last_str}")
    
    # Mostrar períodos no Grupo1
    print("   Períodos no Grupo1:")
    periodos_g1 = df_grupo1[df_grupo1['chave_temp'] == chave][['First_Detected', 'Last_Detected']].head(15)
    for _, p in periodos_g1.iterrows():
        print(f"      {p['First_Detected']} → {p['Last_Detected']}")

# Limpar
df_grupo1 = df_grupo1.drop(columns=['chave_temp'])
df_t0 = df_t0.drop(columns=['chave_temp'])


🔍 DIAGNÓSTICO: INVESTIGANDO GRUPO 1

📊 Chaves que AUMENTARAM linhas: 103,632
📊 Total de linhas extras criadas: 107,785

🔴 Top 10 chaves com mais linhas extras:

1. Chave: 2314393f9cf6b09a44b063f5a8d15e14|Claro TV+|TVOD|...
   t0: 1 linhas → Grupo1: 5 linhas (+ 4)
   Períodos na t0:
      01/10/2022 → 26/07/2024
   Períodos no Grupo1:
      2022-10-01 00:00:00 → NaT
      2022-10-01 00:00:00 → 2024-11-01 00:00:00
      2022-10-01 00:00:00 → 2024-07-19 00:00:00
      2022-10-01 00:00:00 → 2024-09-13 00:00:00
      2022-10-01 00:00:00 → NaT

2. Chave: 2314393f9cf6b09a44b063f5a8d15e14|Claro TV+|TV Everywhere|...
   t0: 1 linhas → Grupo1: 5 linhas (+ 4)
   Períodos na t0:
      01/10/2022 → 26/07/2024
   Períodos no Grupo1:
      2022-10-01 00:00:00 → 2024-07-19 00:00:00
      2022-10-01 00:00:00 → 2024-11-01 00:00:00
      2022-10-01 00:00:00 → NaT
      2022-10-01 00:00:00 → NaT
      2022-10-01 00:00:00 → 2024-09-13 00:00:00

3. Chave: 693263f571b5fb6172ca58a1749daa6d|Hoichoi|Free VOD|.

In [20]:
# ----------------------------------------------------------------------------
# CÉLULA 8: Processamento Grupo 2 - Obras Novas (OTIMIZADO COM DUCKDB)
# ----------------------------------------------------------------------------
print("\n" + "="*70)
print("🔄 PROCESSANDO GRUPO 2: OBRAS NOVAS (NÃO ESTAVAM NA t0)")
print("="*70)

# Query SQL para processar Grupo 2
query_grupo2 = """
WITH 
-- 1. Chaves que estão nos Parquets mas NÃO na t0
chaves_novas AS (
    SELECT DISTINCT p.chave
    FROM df_parquets_reg p
    LEFT JOIN df_t0_reg t ON p.chave = t.chave
    WHERE t.chave IS NULL
),

-- 2. Todas as aparições das chaves novas
aparicoes_novas AS (
    SELECT 
        p.chave,
        p.UID,
        p.Platform_Name,
        p.Package_normalizado as Package,
        p.Channel,
        p.data_ref
    FROM df_parquets_reg p
    INNER JOIN chaves_novas cn ON p.chave = cn.chave
),

-- 3. Detectar gaps usando window function
gaps_detectados AS (
    SELECT 
        chave,
        UID,
        Platform_Name,
        Package,
        Channel,
        data_ref,
        LAG(data_ref) OVER (PARTITION BY chave ORDER BY data_ref) as data_anterior,
        DATEDIFF('day', 
            LAG(data_ref) OVER (PARTITION BY chave ORDER BY data_ref), 
            data_ref
        ) as gap_dias
    FROM aparicoes_novas
),

-- 4. Atribuir período baseado em gaps > 22 dias
periodos_numerados AS (
    SELECT 
        chave,
        UID,
        Platform_Name,
        Package,
        Channel,
        data_ref,
        SUM(CASE WHEN gap_dias > 22 OR gap_dias IS NULL THEN 1 ELSE 0 END) 
            OVER (PARTITION BY chave ORDER BY data_ref 
                  ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as periodo_id
    FROM gaps_detectados
),

-- 5. Agregar por chave + período
periodos_agregados AS (
    SELECT 
        chave,
        periodo_id,
        ANY_VALUE(UID) as UID,
        ANY_VALUE(Platform_Name) as Platform_Name,
        ANY_VALUE(Package) as Package,
        ANY_VALUE(Channel) as Channel,
        MIN(data_ref) as First_Detected,
        MAX(data_ref) as Last_Detected,
        COUNT(*) as num_aparicoes
    FROM periodos_numerados
    GROUP BY chave, periodo_id
)

SELECT 
    UID,
    Platform_Name,
    Package,
    Channel,
    chave,
    First_Detected,
    -- Se último período está recente (>= jul/2025), Last = NULL
    CASE 
        WHEN Last_Detected >= '2025-07-01' THEN NULL
        ELSE Last_Detected
    END as Last_Detected,
    num_aparicoes
FROM periodos_agregados
ORDER BY chave, First_Detected
"""

print("🚀 Executando query DuckDB para obras novas...")
df_grupo2 = con.execute(query_grupo2).fetch_df()

print(f"✅ Grupo 2 processado: {len(df_grupo2):,} linhas geradas")
print(f"   📊 Chaves únicas novas: {df_grupo2['chave'].nunique():,}")
print(f"   📊 UIDs únicos novos: {df_grupo2['UID'].nunique():,}")



🔄 PROCESSANDO GRUPO 2: OBRAS NOVAS (NÃO ESTAVAM NA t0)
🚀 Executando query DuckDB para obras novas...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ Grupo 2 processado: 1,020,718 linhas geradas
   📊 Chaves únicas novas: 774,447
   📊 UIDs únicos novos: 295,322


In [24]:
# ----------------------------------------------------------------------------
# CÉLULA 9: Consolidação Final (CORRIGIDA)
# ----------------------------------------------------------------------------
print("\n" + "="*70)
print("🔗 CONSOLIDANDO RESULTADOS")
print("="*70)

# Juntar Grupo 1 e Grupo 2
df_consolidado = pd.concat([df_grupo1, df_grupo2], ignore_index=True)

print(f"✅ Total de linhas consolidadas: {len(df_consolidado):,}")
print(f"   📊 UIDs únicos: {df_consolidado['UID'].nunique():,}")
print(f"   📊 Chaves únicas: {df_consolidado['chave'].nunique():,}")

# CORREÇÃO: Converter datas corretamente
# Primeiro substituir NaT por None, DEPOIS converter para string
df_consolidado['First_Detected'] = df_consolidado['First_Detected'].apply(
    lambda x: x.strftime('%d/%m/%Y') if pd.notna(x) else ''
)
df_consolidado['Last_Detected'] = df_consolidado['Last_Detected'].apply(
    lambda x: x.strftime('%d/%m/%Y') if pd.notna(x) else ''
)

# Remover coluna auxiliar 'chave'
if 'chave' in df_consolidado.columns:
    df_consolidado = df_consolidado.drop(columns=['chave'])

# Remover coluna Package_normalizado se existir
if 'Package_normalizado' in df_consolidado.columns:
    df_consolidado = df_consolidado.drop(columns=['Package_normalizado'])

print("✅ Formatação final aplicada")


🔗 CONSOLIDANDO RESULTADOS
✅ Total de linhas consolidadas: 1,497,366
   📊 UIDs únicos: 341,671
   📊 Chaves únicas: 1,103,569
✅ Formatação final aplicada


In [25]:
# ----------------------------------------------------------------------------
# CÉLULA 9.5: FILTRO POR EXCLUSÃO DE PLATAFORMAS
# ----------------------------------------------------------------------------
print("\n" + "="*70)
print("🔧 REMOVENDO PLATAFORMAS INDESEJADAS")
print("="*70)

# Lista de plataformas para EXCLUIR
PLATAFORMAS_EXCLUIR = [
    'NBA League Pass',
    'Archivio Luce',
    'FIFA+',
    'DAZN'
]

print(f"🚫 Plataformas a excluir: {len(PLATAFORMAS_EXCLUIR)}")
for plat in PLATAFORMAS_EXCLUIR:
    print(f"   - {plat}")

# Contar antes
linhas_antes = len(df_consolidado)
print(f"\n📊 Linhas ANTES do filtro: {linhas_antes:,}")

# Aplicar filtro de exclusão
df_consolidado = df_consolidado[
    ~df_consolidado['Platform_Name'].isin(PLATAFORMAS_EXCLUIR)
].copy()

linhas_depois = len(df_consolidado)
print(f"📊 Linhas DEPOIS do filtro: {linhas_depois:,}")
print(f"📉 Redução: {linhas_antes - linhas_depois:,} linhas ({(linhas_antes - linhas_depois)/linhas_antes*100:.1f}%)")

# Verificar se cabe no Excel
if linhas_depois > 1048576:
    print(f"\n⚠️  AINDA NÃO CABE NO EXCEL!")
    print(f"   Faltam remover ~{linhas_depois - 1048576:,} linhas")
    print(f"   Considere adicionar mais plataformas à lista de exclusão")
else:
    print(f"\n✅ CABE NO EXCEL! ({linhas_depois:,} < 1,048,576)")


🔧 REMOVENDO PLATAFORMAS INDESEJADAS
🚫 Plataformas a excluir: 4
   - NBA League Pass
   - Archivio Luce
   - FIFA+
   - DAZN

📊 Linhas ANTES do filtro: 1,497,366
📊 Linhas DEPOIS do filtro: 1,162,479
📉 Redução: 334,887 linhas (22.4%)

⚠️  AINDA NÃO CABE NO EXCEL!
   Faltam remover ~113,903 linhas
   Considere adicionar mais plataformas à lista de exclusão


In [26]:
# ----------------------------------------------------------------------------
# CÉLULA DE DIAGNÓSTICO: ANÁLISE DE PERÍODOS
# ----------------------------------------------------------------------------
print("\n" + "="*70)
print("🔍 DIAGNÓSTICO: ANALISANDO DISTRIBUIÇÃO DE PERÍODOS")
print("="*70)

# Recriar chave temporariamente
df_consolidado['chave_temp'] = (
    df_consolidado['UID'].astype(str) + '|' + 
    df_consolidado['Platform_Name'].astype(str) + '|' + 
    df_consolidado['Package'].fillna('').astype(str) + '|' + 
    df_consolidado['Channel'].fillna('').astype(str)
)

# 1. Quantos períodos por chave?
periodos_por_chave = df_consolidado.groupby('chave_temp').size()

print("\n### DISTRIBUIÇÃO DE PERÍODOS POR CHAVE ###")
print(periodos_por_chave.describe())
print("\n📊 Contagem de períodos (quantas chaves têm X períodos):")
contagem = periodos_por_chave.value_counts().sort_index().head(20)
for num_periodos, qtd_chaves in contagem.items():
    print(f"   {num_periodos} período(s): {qtd_chaves:,} chaves ({qtd_chaves/len(periodos_por_chave)*100:.1f}%)")

# 2. Chaves com MUITOS períodos (possível problema)
chaves_suspeitas = periodos_por_chave[periodos_por_chave > 5]
if len(chaves_suspeitas) > 0:
    print(f"\n⚠️  CHAVES COM >5 PERÍODOS: {len(chaves_suspeitas):,} ({len(chaves_suspeitas)/len(periodos_por_chave)*100:.1f}%)")
    print("\nExemplos (top 10 com mais períodos):")
    for chave, count in chaves_suspeitas.sort_values(ascending=False).head(10).items():
        print(f"\n   🔴 {count} períodos: {chave[:80]}...")
        
        # Mostrar as datas desses períodos
        periodos_exemplo = df_consolidado[df_consolidado['chave_temp'] == chave][
            ['UID', 'Platform_Name', 'First_Detected', 'Last_Detected']
        ].head(15)
        for idx, row in periodos_exemplo.iterrows():
            print(f"      {row['First_Detected']} → {row['Last_Detected']}")

# 3. Análise de obras abertas vs fechadas
print("\n### ANÁLISE TEMPORAL ###")
total = len(df_consolidado)
abertas = (df_consolidado['Last_Detected'] == '').sum()
fechadas = total - abertas
print(f"Obras ABERTAS (Last vazio):       {abertas:,} ({abertas/total*100:.1f}%)")
print(f"Obras FECHADAS (Last preenchido): {fechadas:,} ({fechadas/total*100:.1f}%)")

# 4. Contagem por grupo de origem
print("\n### ORIGEM DOS DADOS ###")
if 'origem_grupo' in df_consolidado.columns:
    print(df_consolidado['origem_grupo'].value_counts())

# Remover chave temporária
df_consolidado = df_consolidado.drop(columns=['chave_temp'])


🔍 DIAGNÓSTICO: ANALISANDO DISTRIBUIÇÃO DE PERÍODOS

### DISTRIBUIÇÃO DE PERÍODOS POR CHAVE ###
count    878008.000000
mean          1.323996
std           0.536936
min           1.000000
25%           1.000000
50%           1.000000
75%           2.000000
max          37.000000
dtype: float64

📊 Contagem de períodos (quantas chaves têm X períodos):
   1 período(s): 617,881 chaves (70.4%)
   2 período(s): 239,268 chaves (27.3%)
   3 período(s): 18,503 chaves (2.1%)
   4 período(s): 1,683 chaves (0.2%)
   5 período(s): 424 chaves (0.0%)
   6 período(s): 166 chaves (0.0%)
   7 período(s): 52 chaves (0.0%)
   8 período(s): 13 chaves (0.0%)
   9 período(s): 6 chaves (0.0%)
   10 período(s): 2 chaves (0.0%)
   11 período(s): 1 chaves (0.0%)
   12 período(s): 4 chaves (0.0%)
   15 período(s): 1 chaves (0.0%)
   16 período(s): 1 chaves (0.0%)
   17 período(s): 1 chaves (0.0%)
   19 período(s): 1 chaves (0.0%)
   37 período(s): 1 chaves (0.0%)

⚠️  CHAVES COM >5 PERÍODOS: 249 (0.0%)

Exemplos 

In [ ]:
# ----------------------------------------------------------------------------
# CÉLULA 10: Export para PARQUET e EXCEL
# ----------------------------------------------------------------------------
print("\n" + "="*70)
print("💾 EXPORTANDO RESULTADOS")
print("="*70)

# Gerar nome base com timestamp
timestamp = datetime.now().strftime('%Y-%m-%d')
nome_base = f'Base_Consolidada_t0_t1_{timestamp}'

# 1. SALVAR PARQUET (sempre funciona, sem limite de linhas)
arquivo_parquet = PASTA_OUTPUT / f'{nome_base}.parquet'
df_consolidado.to_parquet(arquivo_parquet, index=False)
print(f"✅ Parquet salvo: {arquivo_parquet}")
print(f"   📊 Dimensões: {len(df_consolidado):,} linhas x {len(df_consolidado.columns)} colunas")

# 2. SALVAR EXCEL (se couber)
arquivo_excel = PASTA_OUTPUT / f'{nome_base}.xlsx'

if len(df_consolidado) <= 1048576:
    df_consolidado.to_excel(arquivo_excel, index=False)
    print(f"✅ Excel salvo: {arquivo_excel}")
else:
    print(f"⚠️  Excel NÃO salvo - arquivo muito grande ({len(df_consolidado):,} linhas)")
    print(f"   Arquivo Parquet disponível: {arquivo_parquet}")
    print(f"   Adicione mais plataformas à exclusão na célula 9.5")

In [ ]:
# ----------------------------------------------------------------------------
# CÉLULA 11: Relatório de Auditoria
# ----------------------------------------------------------------------------
print("\n" + "="*70)
print("📊 RELATÓRIO DE AUDITORIA")
print("="*70)

print("\n### ORIGEM DOS DADOS ###")
print(f"Grupo 1 (da t0):     {len(df_grupo1):,} linhas")
print(f"Grupo 2 (novas):     {len(df_grupo2):,} linhas")
print(f"TOTAL:               {len(df_consolidado):,} linhas")

print("\n### ANÁLISE TEMPORAL ###")
obras_abertas = (df_consolidado['Last_Detected'] == '').sum()
obras_fechadas = (df_consolidado['Last_Detected'] != '').sum()
print(f"Obras ABERTAS (Last vazio):    {obras_abertas:,} ({obras_abertas/len(df_consolidado)*100:.1f}%)")
print(f"Obras FECHADAS (Last preenchido): {obras_fechadas:,} ({obras_fechadas/len(df_consolidado)*100:.1f}%)")

print("\n### DISTRIBUIÇÃO POR PLATAFORMA (Top 10) ###")
print(df_consolidado['Platform_Name'].value_counts().head(10))

print("\n### DISTRIBUIÇÃO POR PACKAGE ###")
print(df_consolidado['Package'].value_counts())

print("\n" + "="*70)
print(f"✅ PROCESSAMENTO CONCLUÍDO EM: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*70)

# Fechar conexão DuckDB
con.close()